# Pedro - Short Queeze Predictor
---

### 1. Libraries Import

In [6]:
import os
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

import xgboost as xgb
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

### 2. Alpaca API Setup

In [7]:
def create_alpaca_api():
    load_dotenv(dotenv_path="Alpaca.env")

    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    alpaca_api_base_url = "https://paper-api.alpaca.markets"

    api = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        alpaca_api_base_url,
        api_version="v2"
    )

    return api

In [8]:
api = create_alpaca_api()

In [9]:
# Get account information
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Print our current balance.
print('${} is available as buying power.'.format(account.buying_power))

$200000 is available as buying power.


### 3. Load Models

In [10]:
def load_models(model_names, folder_path='Models'):
    """
    Loads trained models based on the given model names from the specified folder.
    """
    models = {}
    for name in model_names:
        # Load each model from its file in the 'Models' folder
        models[name] = joblib.load(os.path.join(folder_path, f"{name.replace(' ', '_')}.joblib"))
        
    return models

print("Function 'load_models' defined.")

Function 'load_models' defined.


In [11]:
# Load models
model_names = ["Gradient_Boosting_Classifier", "Support_Vector_Classifier", 
               "Random_Forest_Classifier", "Decision_Tree_Classifier", 
               "XGBoost_Classifier", "Neural_Network"]

models = load_models(model_names)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-07-13 12:10:00         1835
metadata.json                                  2023-07-13 12:10:00           64
variables.h5                                   2023-07-13 12:10:00        36264
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [ ]:
# Set signal variable
signal = 1

In [ ]:
# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"

In [ ]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [ ]:
# Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Get final closing price
limit_amount = prices["TSLA"]["close"][-1]

In [ ]:
# Submit order
api.submit_order(
    symbol="TSLA", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="gtc", 
    type="limit", 
    limit_price=limit_amount
)